In [32]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as sp_sum, array, concat_ws, regexp_replace, explode, split


# Spark 세션 생성
spark = SparkSession \
    .builder \
    .appName("PySparkTest") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()


# csv 경로
csv_path = '/Users/b06/Desktop/yeardream/medi-05/data/meta_sample.csv'


# 데이터 로드
df = spark.read.csv(json_path, header=True)

In [33]:
df.printSchema()

root
 |-- [{"ROOT_QUERY": {"__typename": "Query": string (nullable = true)
 |--  "user": {"__typename": "UserResult"1: string (nullable = true)
 |--  "partnerHashedIdNo": null2: string (nullable = true)
 |--  "myplace": {"__typename": "MyPlaceUserInfo"3: string (nullable = true)
 |--  "profile": {"__typename": "MyPlaceProfile"4: string (nullable = true)
 |--  "imageUrl": null5: string (nullable = true)
 |--  "borderImageUrl": null}}}6: string (nullable = true)
 |--  "accessor": {"__typename": "Accessor"7: string (nullable = true)
 |--  "accessor": ""8: string (nullable = true)
 |--  "accessorBanner": []}9: string (nullable = true)
 |--  "hospital": null: string (nullable = true)
 |--  "panoramaThumbnail": {"__typename": "PanoramaThumbnailResult"11: string (nullable = true)
 |--  "url": "https://apis.naver.com/place/panorama/thumbnail/11779766414141/0?width=800&height=400&msgpad=1696472191134&md=IUhTuI2%2B78CNjKUAfl9NvPkhnYc%3D"}}}: string (nullable = true)
 |--  {"Panorama:bydPStvfMsdI

In [35]:
### 필요 없는 칼럼들은 제외


# 특정 컬럼들만 추출
cols_selected = [
    'review_settings_keyword', 
    'keywords_1', 'keywords_2', 'keywords_3', 'keywords_4', 'keywords_5', 
    'conveniences_1', 'conveniences_2', 'conveniences_3', 'conveniences_4', 'conveniences_5', 
    'conveniences_6', 'conveniences_7', 'conveniences_8', 'conveniences_9', 'conveniences_10', 
    'conveniences_11',     
    'description',
    'id', 'name', 'visitor_reviews_total', 'photo_review_ratio', 'fsas_reviews_total', 
    'images_count', 'homepages_type', 'homepages_isDeadUrl', 'homepages_isRep',
    'description_length', 'self_blog_present', 'qna_answer_count', 'crawled_at',    
]


# 특정 컬럼만 추출
df_selected = df.select(*cols_selected)


# 추출된 데이터 확인
df_selected.show(truncate=False)

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `review_settings_keyword` cannot be resolved. Did you mean one of the following? [` "isKtis": null`, ` "avgRating": 0`, ` "cescoLink": null`, ` "isPostExists": true`, ` "postList": []}`].;
'Project ['review_settings_keyword, 'keywords_1, 'keywords_2, 'keywords_3, 'keywords_4, 'keywords_5, 'conveniences_1, 'conveniences_2, 'conveniences_3, 'conveniences_4, 'conveniences_5, 'conveniences_6, 'conveniences_7, 'conveniences_8, 'conveniences_9, 'conveniences_10, 'conveniences_11, 'description, 'id, 'name, 'visitor_reviews_total, 'photo_review_ratio, 'fsas_reviews_total, 'images_count, ... 7 more fields]
+- Relation [[{"ROOT_QUERY": {"__typename": "Query"#9307, "user": {"__typename": "UserResult"1#9308, "partnerHashedIdNo": null2#9309, "myplace": {"__typename": "MyPlaceUserInfo"3#9310, "profile": {"__typename": "MyPlaceProfile"4#9311, "imageUrl": null5#9312, "borderImageUrl": null}}}6#9313, "access

In [19]:
### 전처리에 필요한 함수들을 생성
    

def replace_string(df, column_name, pattern, replacement):
    """특정 컬럼의 문자열을 치환하는 함수"""
    return df.withColumn(column_name, regexp_replace(col(column_name), pattern, replacement))
    

def explode_column(df, column_name):
    """특정 컬럼을 분리하고 펼치는 함수"""
    return df.withColumn(column_name, explode(split(col(column_name), ", ")))


def df_id_and_combined_column(df, prefix):
    """특정 접두사를 가진 칼럼들을 하나의 문자열 칼럼으로 합치는 함수"""
    cols = [c for c in df.columns if c.startswith(prefix + "_")]
    cols_arr = array([col(c) for c in cols]).alias(prefix)])
    table = df.select("id", cols_arr)
    return table


def concat_and_explode_column(df, column_name):
    """특정 array 컬럼을 문자열로 변환하고 분리하여 펼치는 함수"""
    return df.withColumn(column_name, explode(split(concat_ws(", ", column_name), ", ")))

SyntaxError: unmatched ']' (1722637305.py, line 17)

In [20]:
# Table 1: id, review_settings_keyword
table1 = df_selected.select("id", "review_settings_keyword")
table1 = replace_string(table1, "review_settings_keyword", "[()]", "")  # 중괄호와 대괄호는 삭제함
table1 = replace_string(table1, "review_settings_keyword", " & ", ", ") # & -> , -> 제1 정규화
table1 = explode_column(table1, "review_settings_keyword")
table1.show()

+----------+--------------------------------+
|        id|         review_settings_keyword|
+----------+--------------------------------+
|  19518309|                      송정한의원|
|  19518309|   성동구 | 송정동 | 한의원-일반|
|  19518309|                      송정한의원|
|  20709457|               용산구 소망한의원|
|  19523171|                금호호랑이한의원|
|  19523171|성동구 | 금호동3가 | 한의원-일반|
|  19523171|                금호호랑이한의원|
|  18757770|                      약촌한의원|
|  18757770|   서초구 | 잠원동 | 한의원-일반|
|  18757770|                      약촌한의원|
|1079161835|                      장수한의원|
|1079161835|   종로구 | 창신동 | 한의원-일반|
|1079161835|                      장수한의원|
|1703239053|                  성동예본한의원|
|1703239053|   성동구 | 행당동 | 한의원-일반|
|1703239053|                  성동예본한의원|
|  19529164|                      태평한의원|
|  19529164|     중구 | 신당동 | 한의원-일반|
|  19529164|                      태평한의원|
| 230007709|                    알파스한의원|
+----------+--------------------------------+
only showing top 20 rows



In [21]:
# Table 2: id, keywords
prefix = "keyword"
table2 = df_id_and_combined_column(df_selected, prefix)
table2 = concat_and_explode_column(table2, prefix)
table2.show()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `keyword` cannot be resolved. Did you mean one of the following? [`keywor`, `id`].;
'Project [id#17, keywor#3595, explode(split(concat_ws(, , 'keyword), , , -1)) AS keyword#3598]
+- Project [id#17, array(keywords_1#20, keywords_2#21, keywords_3#22, keywords_4#23, keywords_5#24) AS keywor#3595]
   +- Project [review_settings_keyword#19, keywords_1#20, keywords_2#21, keywords_3#22, keywords_4#23, keywords_5#24, conveniences_1#41, conveniences_2#42, conveniences_3#43, conveniences_4#44, conveniences_5#45, conveniences_6#46, conveniences_7#47, conveniences_8#48, conveniences_9#49, conveniences_10#50, conveniences_11#51, description#59, id#17, name#18, visitor_reviews_total#29, photo_review_ratio#30, fsas_reviews_total#31, images_count#32, ... 7 more fields]
      +- Relation [id#17,name#18,review_settings_keyword#19,keywords_1#20,keywords_2#21,keywords_3#22,keywords_4#23,keywords_5#24,menu_name#

In [8]:
# Table 3: id, conveniences
prefix = "conveniences"
table3 = df_id_and_combined_column(df_selected, prefix)
table3 = concat_and_explode_column(table3, prefix)
table3.show()

+----------+-----------------+
|        id|     conveniences|
+----------+-----------------+
|  19518309|                 |
|  20709457|                 |
|  19523171|             주차|
|  19523171|      무선 인터넷|
|  18757770|                 |
|1079161835|                 |
|1703239053|             주차|
|1703239053|남/녀 화장실 구분|
|  19529164|                 |
| 230007709|                 |
|  19518394|                 |
|  19529128|                 |
|  36523267|             주차|
|  36523267|             예약|
|  36523267|남/녀 화장실 구분|
|  36523267|  장애인 편의시설|
|  13100633|                 |
|  19518670|                 |
|1524076456|             주차|
|1524076456|             예약|
+----------+-----------------+
only showing top 20 rows



In [10]:
# Table 4: id, description
table4 = df_selected.select("id", "description")
table4.show()

+----------+------------------------------------+
|        id|                         description|
+----------+------------------------------------+
|  19518309|                                NULL|
|  20709457|                                NULL|
|  19523171|  대기시간을 최소화하기위해 방문 ...|
|  18757770|                                NULL|
|1079161835|    안녕하세요. 늘 환자의 건강을 ...|
|1703239053|      성동예본한의원은 2022년 7월...|
|  19529164|                                NULL|
| 230007709|                                NULL|
|  19518394|                                NULL|
|  19529128|                                NULL|
|  36523267|    대한민국365한의원은 산재 지정...|
|  13100633|                                NULL|
|  19518670|                                NULL|
|1524076456|   안녕하세요! 비수술 척추관절 추...|
|  19527635|                                NULL|
|  13153787|                                NULL|
|  13021125|여성비만만성피로비염통증치료등 한...|
|  19529179|                                NULL|
|  18049532|               

In [12]:
# Table 5: id와 나머지 모든 컬럼
exclude_cols = [
    'review_settings_keyword',
    'keywords_1', 'keywords_2', 'keywords_3', 'keywords_4', 'keywords_5',
    'conveniences_1', 'conveniences_2', 'conveniences_3', 'conveniences_4', 'conveniences_5',
    'conveniences_6', 'conveniences_7', 'conveniences_8', 'conveniences_9', 'conveniences_10',
    'conveniences_11',
    'description'
]
table5_cols = [col for col in df_selected.columns if col not in exclude_cols]
table5 = df_selected.select(*table5_cols)
table5.show()

+----------+-----------------+---------------------+------------------+------------------+------------+--------------+-------------------+---------------+------------------+-----------------+----------------+--------------------+
|        id|             name|visitor_reviews_total|photo_review_ratio|fsas_reviews_total|images_count|homepages_type|homepages_isDeadUrl|homepages_isRep|description_length|self_blog_present|qna_answer_count|          crawled_at|
+----------+-----------------+---------------------+------------------+------------------+------------+--------------+-------------------+---------------+------------------+-----------------+----------------+--------------------+
|  19518309|       송정한의원|                   22| 0.409090909090909|                 0|           2|          NULL|               NULL|           NULL|                 0|             NULL|            NULL|2023-10-05 00:00:...|
|  20709457|       소망한의원|                   22| 0.681818181818182|                 0|

In [13]:
data_path = '/Users/b06/Desktop/yeardream/medi-05/data/pyspark-test'
table1_path = f"{data_path}/review_settings_keyword"
table2_path = f"{data_path}/keywords"
table3_path = f"{data_path}/conveniences"
table4_path = f"{data_path}/description"
table5_path = f"{data_path}/data"

In [14]:
def save(table, table_path):
    table\
        .write \
        .mode('overwrite') \
        .option("header", "true") \
        .option("charset", "cp949") \
        .csv(table_path)

In [15]:
save(table1, table1_path)
save(table2, table2_path)
save(table3, table3_path)
save(table4, table4_path)
save(table5, table5_path)